In [1]:
###
#Use Tensornet_carmelo_npt codebase
###
import pytest
import pytorch_lightning as pl

from matsciml.datasets.transforms import (
    PeriodicPropertiesTransform,
    PointCloudToGraphTransform,
    PointCloudToGraphTransform,
    MGLDataTransform,
)

from matsciml.lightning import MatSciMLDataModule
from matsciml.common.registry import registry
from matsciml.models import TensorNet
from matsciml.datasets.utils import element_types
from matsciml.models.base import ForceRegressionTask, GradFreeForceRegressionTask

import torch

In [2]:
from matsciml.models.utils.io import * 
checkpoint_path = "/home/m3rg2000/Simulation/checkpoints-2024/epoch=4-step=4695_tensornet_force_r.ckpt"
Loaded_model = multitask_from_checkpoint(checkpoint_path)

In [3]:
from matsciml.preprocessing.atoms_to_graphs import *
a2g=AtomsToDGL(max_neigh=50,
            radius=6,
            r_energy=False,
            r_forces=False,
            r_distances=False,
            r_edges=True,
            r_fixed=True,)


from matsciml.datasets.trajectory_lmdb import data_list_collater

PBCTransform=PeriodicPropertiesTransform(cutoff_radius=6.0, adaptive_cutoff=True)
GTransform=PointCloudToGraphTransform(
                "dgl",
                node_keys=["pos", "atomic_numbers"],
            )

def convAtomstoBatch(atoms):
    data_obj=a2g.convert(atoms)
    Reformatted_batch={
        'cell' : data_obj.cell,
        'natoms' :  torch.Tensor([data_obj.natoms]).unsqueeze(0),
        'edge_index' : data_obj.edge_index,
        'cell_offsets': data_obj.cell_offsets,
        'atomic_numbers': data_obj.atomic_numbers,
        'y' : None,
        'pos' : data_obj.pos,
        'force' : None, 
        'fixed' : [data_obj.fixed],
        'tags' : None,
        'sid' :None,
        'fid' : None,
        'dataset' : 'S2EFDataset',
        #'graph' : data_obj.graph,
    }
    Reformatted_batch=PBCTransform(Reformatted_batch)
    Reformatted_batch=GTransform(Reformatted_batch)
    return Reformatted_batch

In [4]:
from matsciml.datasets.trajectory_lmdb import data_list_collater
from __future__ import annotations

import copy
import logging
import os

import torch
import yaml
from ase import Atoms
from ase.calculators.calculator import Calculator
from ase.calculators.singlepoint import SinglePointCalculator as sp
from ase.constraints import FixAtoms

from matsciml.common.registry import registry
from matsciml.common.utils import radius_graph_pbc, setup_imports, setup_logging
from matsciml.datasets.trajectory_lmdb import data_list_collater
from torch_geometric.data import Batch
from ase import units
from ase.md.langevin import Langevin
from ase.io import read, write
import numpy as np
import time
import torch
import sys
from tqdm import tqdm

import time


In [42]:
import torch
from ase.calculators.calculator import Calculator, all_changes
#from ase.stress import full_3x3_to_voigt_6_stress

class TensorNet_ASEcalculator(Calculator):
    """Simulation ASE Calculator"""

    implemented_properties = ["energy" , "forces","stress"]

    def __init__(
        self,
        model,
        **kwargs
    ):
        Calculator.__init__(self, **kwargs)
        self.results = {}
        self.model = model
        
        
    # pylint: disable=dangerous-default-value
    def calculate(self, atoms=None, properties=None, system_changes=all_changes):
        """
        Calculate properties.
        :param atoms: ase.Atoms object
        :param properties: [str], properties to be computed, used by ASE internally
        :param system_changes: [str], system changes since last calculation, used by ASE internally
        :return:
        """
        # call to base-class to set atoms attribute
        Calculator.calculate(self, atoms)

        # prepare data
        batch=convAtomstoBatch(atoms)

        # predict + extract data
        out = self.model.forward(batch)
        #print(out)
        energy = out['force_regression0']["energy"].detach().cpu().item()
        forces = out['force_regression0']["force"].detach().cpu().numpy()
        stress = out['force_regression0']["stress"].squeeze(0).detach().cpu().numpy()
        # store results
        E = energy
        stress= np.array([stress[0, 0],
                                   stress[1, 1],
                                   stress[2, 2],
                                   stress[1, 2],
                                   stress[0, 2],
                                   stress[0, 1]])
        self.results = {
            "energy": E,
            # force has units eng / len:
            "forces": forces,
            "stress" : stress,
        }


In [43]:
from ase import units
from ase.md.langevin import Langevin
from ase.io import read, write
import numpy as np
import time
import torch
import argparse
import sys
from tqdm import tqdm
import time
from ase.io import read


start_time=time.time()

calculator = TensorNet_ASEcalculator(Loaded_model)
#calculator.model.double() # change model weights type to double precision(hack to avoid error)
# Path to the CIF file
#cif_file_path = "/home/m3rg2000/Simulation/checkpoints-2024/S2EF.extxyz"

cif_file_path = "/home/m3rg2000/Simulation/checkpoints-2024/Example.cif"

#cif_file_path = "/home/m3rg2000/Simulation/checkpoints-2024/SiO2.cif"

# Read CIF file using ASE
atoms = read(cif_file_path)
init_conf=atoms
init_conf.set_calculator(calculator)
#calculator.model.double()
dyn = Langevin(init_conf, 1*units.fs, temperature_K=298,friction=5e-3) 
def write_frame():
    dyn.atoms.write(f'md_TensorNET_NPT_SiO2_CIF.xyz', append=True)
dyn.attach(write_frame, interval=1)
dyn.run(100)
print(f"MD finished!")

KeyboardInterrupt: 

In [7]:
atoms.get_cell_lengths_and_angles()

/home/m3rg2000/miniconda3/envs/matsciml/lib/python3.9/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


array([  5.268,   5.268,  13.48 ,  90.   ,  90.   , 120.   ])

In [8]:
from ase import Atoms
from ase.units import Angstrom

total_mass = atoms.get_masses().sum()  # in atomic mass units (amu)
volume = atoms.get_volume()  # in cubic Angstroms
total_mass_kg = total_mass * 1.6605402e-27
volume_m3 = volume * ((1e-10) ** 3)
density = total_mass_kg / volume_m3 * 1e-3  # in g/cc

print(f'Density: {density:.10f} g/cc')


Density: 5.8779871720 g/cc


In [9]:
Simulation_Data=read("/home/m3rg2000/Simulation/Scritps/md_TensorNET_NPT_SiO2_CIF.xyz",index=':')

In [33]:
Densities=[]
Cell_params=[]
for atoms in Simulation_Data:
    total_mass = atoms.get_masses().sum()  # in atomic mass units (amu)
    volume = atoms.get_volume()  # in cubic Angstroms
    total_mass_kg = total_mass * 1.6605402e-27
    volume_m3 = volume * ((1e-10) ** 3)
    density = total_mass_kg / volume_m3 * 1e-3  # in g/cc
    Densities+=[density]
    cell=atoms.get_cell_lengths_and_angles()
    Cell_params+=[cell]
    #print(f'Density: {density:.10f} g/cc')
Densities=torch.Tensor(Densities)
Cell_params_av=torch.Tensor(Cell_params).mean(dim=0).tolist()
print("Predicted Density: ",f'{Densities.mean().item():.4f}', "g/cc")
print(f"a: {Cell_params_av[0]:.3f}\nb: {Cell_params_av[1]:.3f}\nc: {Cell_params_av[2]:.3f}")
print(f"alpha: {Cell_params_av[3]:.3f}\nbeta: {Cell_params_av[4]:.3f}\ngamma: {Cell_params_av[5]:.3f}")

Predicted Density:  5.8780 g/cc
Lattice Params:
a: 5.268
b: 5.268
c: 13.480
alpha: 90.000
beta: 90.000
gamma: 120.000


In [45]:
#For NPT Simulation
from ase.md import Langevin
from ase.md.nptberendsen import NPTBerendsen
 
# Define the temperature and pressure
temperature_K = 298
pressure = 1.0  # in atm

calculator = TensorNet_ASEcalculator(Loaded_model)
atoms = read(cif_file_path)
init_conf=atoms
init_conf.set_calculator(calculator)
# Initialize the NPT dynamics
dyn = NPTBerendsen(init_conf, timestep=0.1 * units.fs, temperature_K=300,
                   taut=100 * units.fs, pressure_au=1.01325 * units.bar,
                   taup=1000 * units.fs, compressibility_au=4.57e-5 / units.bar)
def write_frame():
    dyn.atoms.write('md_FAENET_npt2.xyz', append=True)

dyn.attach(write_frame, interval=1)
dyn.run(1000)

print("NPT Simulation Finished!")


NPT Simulation Finished!


In [40]:
atoms.get_stress()

array([ 1.91892788e-04,  1.75146066e-04,  5.27283919e-05,  7.68272059e-07,
        8.27364147e-06, -9.84892631e-06])